# Parsing csv to xlsx
## Data Sources
https://api.ehealth.gov.ua/api/services
## Commit git
- 3ed8a2e Start


In [30]:
import pandas as pd
import re

In [57]:
pd.read_json('./data/interim/test2.json', orient='records')

,code,id,inserted_at,inserted_by,is_active,name,request_allowed,services,updated_at,updated_by
0,K41993,f3ec9dbd-dce5-4a86-b21a-7b4fff03ac1e,2020-03-31 19:52:32.645167+00:00,4261eacf-8008-4e62-899f-de1e2f7065f0,True,Цифрова субтракційна ангіографія грудної клітки,True,"[{'category': 'imaging', 'code': '60012-00', '...",2020-03-31 19:52:32.645167+00:00,4261eacf-8008-4e62-899f-de1e2f7065f0
1,L41985,f6f10ba1-3674-4808-afa2-ba30e65d2a37,2020-03-31 19:52:32.645167+00:00,4261eacf-8008-4e62-899f-de1e2f7065f0,True,Артрографія,True,"[{'category': 'imaging', 'code': '59751-00', '...",2020-03-31 19:52:32.645167+00:00,4261eacf-8008-4e62-899f-de1e2f7065f0
